In [1]:
import os, json, time
from gridappsd import GridAPPSD, topics as t
from gridappsd.topics import service_input_topic, service_output_topic
from distributedtopology import DistributedTopology
from topologydictionary import TopologyDictionary
from networkmodel import NetworkModel

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

# Connect to GridAPPS-D Platform
gapps = GridAPPSD()
assert gapps.connected



In [2]:
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA"

#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F"  # 13 assets
#model_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507"  # 123 bus

In [3]:
Topology = TopologyDictionary(gapps, model_mrid)


In [4]:
network = NetworkModel(gapps)


In [5]:
network.build_equip_dicts(model_mrid, Topology)

In [6]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine', 'PowerElectronicsConnection']
Topology.build_linknet(EqTypes)


In [ ]:
Topology.update_switches()


In [ ]:
Topology.build_feeder_islands()

In [ ]:
i1 = 0
root_type = list(Topology.Feeders['feeder_'+ str(i1)].keys())[1]
root_id = Topology.Feeders['feeder_'+ str(i1)][root_type]
root_node = Topology.Feeders['feeder_'+ str(i1)]['root_node']


In [7]:
Topology.ConnNodeDict['_F326CDF0-B928-416B-8D96-C3DA1ADE7522']

{'name': 'm2001der480-2',
 'TopologicalNode': '_CD9DE392-495C-4EB9-B55C-CD6C2A1A75D9',
 'nominalVoltage': '480',
 'ACLineSegment': ['_93611BCF-1948-4089-BDA4-C5B7B0582C84',
  '_4C7BDE40-AA4C-4FD6-919F-111D2F063517'],
 'Breaker': [],
 'EnergyConsumer': [],
 'Fuse': [],
 'House': [],
 'LinearShuntCompensator': [],
 'LoadBreakSwitch': [],
 'PowerTransformer': ['_306F9616-8535-4713-B58E-B0661F790651'],
 'RatioTapChanger': [],
 'Recloser': [],
 'TransformerTank': [],
 'SynchronousMachine': [],
 'PowerElectronicsConnection': [],
 'Measurement': ['_2f3dea5e-add4-4ffc-988b-f91f3f87611c',
  '_b56ccec6-01c7-484b-8082-d31897bfe05c',
  '_cb0541c5-37ea-4d2a-ae9b-6b25ef2375ae',
  '_3afd2249-c5ec-41ad-9090-55ffb0b77317',
  '_438799e9-e50e-45cf-b97a-8c8299904d2d',
  '_97462b6f-74c3-4b44-ab86-624fb3eff48e',
  '_15d84852-be2e-45c1-a175-2d294a06dc18',
  '_594d348f-32a7-4b3c-8536-e2da605caf63',
  '_d11f252a-fc85-4b4a-8353-475c63ed604a',
  '_6015aa62-9b70-468b-83aa-8c9aecabf0ad',
  '_7df73a67-7e24-4113-a77

In [ ]:
Topology.EquipDict[root_type][root_id]

In [ ]:
Topology.EquipDict['PowerElectronicsConnection']

In [10]:
Topology.ConnNodeDict[5227]

KeyError: 5227